In [95]:
import numpy as np
from clarkewright import *

### Step 1 

In [2]:
#초기 거리 값  
# full_mat is travel_time_matrix of from vendor to vendor 
f = [0,1.6,0.6,2,1.7,1.6,1.5],[1.6,0,1.9,3.5,0.7,1.3,3.1],[0.6,1.9,0,1.6,2.2,2.2,1.6],[2,3.5,1.6,0,3.7,3.6,1.4],[1.7,0.7,2.2,3.7,0,0.7,3.1],[1.6,1.3,2.2,3.6,0.7,0,2.7], [1.5,3.1,1.6,1.4,3.1,2.7,0]
full_mat = np.array(f)
print(full_mat)
print(full_mat.shape)

[[0.  1.6 0.6 2.  1.7 1.6 1.5]
 [1.6 0.  1.9 3.5 0.7 1.3 3.1]
 [0.6 1.9 0.  1.6 2.2 2.2 1.6]
 [2.  3.5 1.6 0.  3.7 3.6 1.4]
 [1.7 0.7 2.2 3.7 0.  0.7 3.1]
 [1.6 1.3 2.2 3.6 0.7 0.  2.7]
 [1.5 3.1 1.6 1.4 3.1 2.7 0. ]]
(7, 7)


In [3]:
###Input 인수들  
# number of vehicle = 1
D = [1000,200,300,400,50,600,700] #Weekly demand of item i
C = [10,60,30,400,4000,200,40] #Unit cost of item i
Cr = 0.01 #Inventory holding cost (% of unit cost/week)
Cp = [0,0,0,0,0,0,0] #Setup cost to pick up item i
VVC = 12 #Variable vehicle cost ($/hour of travel time)
P = [[1],[2],[3],[4],[5],[6,7]] #Set of all items to be picked up from vendor k

### Step 2
- set alpha to zero  
- calculate dtmax(k), dtmin(k), nmax(k), nmin(k)  
    - dtmax(k) = upper bound on the travel time of vendor k  
               = full_mat(plant to vendor k) + full_mat(vendor k to plant)   
    - dtmin(k) = lower bound on the travel time of vendor k    
               = full_mat(closest vendor of k) + full_mat(next closest vendor of k)  
    - n(k) = minimum-cost number of pickups of vendor k (== # of orders)  
    - nmin(k) = lower bound of number of pickups of vendor k  
    - nmax(k) = upper bound of number of pickups of vendor k   

In [14]:
# 두 번째로 작은 값 출력 함수
def second_smallest(numbers):
    m1, m2 = 10000,10000
    for x in numbers:
        if x <= m1:
            m1, m2 = x, m1
        elif x < m2:
            m2 = x  
    return m2

In [15]:
# 세 번째로 작은 값 출력 함수
def third_smallest(numbers):
    m1, m2, m3 = 10000,10000, 10000
    for x in numbers:
        if x <= m1:
            m1, m2, m3 = x, m1, m2
        elif x < m2:
            m2, m3 = x, m2
        elif x < m3:
            m3 = x
    return m3

In [16]:
#dt_min값 / dt_max 계산 함수_second_smallest / third_smallest 함수 사용
def cal_dtmin_dtmax(n):
   
    full_mat2 = np.copy(full_mat[1:,1:])
    full_mat3 = np.copy(full_mat[1:,:])

    dt_min, dt_max, dtmin1, dtmin2, dtmin, dtmax = [],[],[],[],[],[]  
    
    for k in range(n-1):
        dtmin.append(10000)
        dtmin[k] = second_smallest(full_mat2[k]) + third_smallest(full_mat2[k])        
        dt_min.append(dtmin[k])
        
    for i in range(n-1):
        dtmax.append(0)
        dtmax[i] += full_mat3[i][0] * 2
        if dtmax[i] < dtmin[i]:
            dtmax[i] = dtmin[i]
        dt_max.append(dtmax[i])
    
    return dt_min, dt_max

In [12]:
#dt_min, dt_max 구하기
dt_min, dt_max = cal_dtmin_dtmax(len(full_mat))
print(dt_min)
print(dt_max)

[2.0, 3.2, 3.0, 1.4, 2.0, 3.0]
[3.2, 3.2, 4.0, 3.4, 3.2, 3.0]


In [17]:
#n_min, n_max 계산 함수
def cal_nmax_nmin():
    n_max,n_min = [], []
    for k in range(len(P)):
        sum1 = 0
        sum2 = 0
        for i in P[k]:  
            sum1 += D[i-1]*C[i-1]
            sum2 += Cp[i-1]
        a = (Cr*sum1)/(2*(sum2+VVC*dt_min[k]))
        b = (Cr*sum1)/(2*(sum2+VVC*dt_max[k]))
        n_max.append(round(a**0.5,2))
        n_min.append(round(b**0.5,2))
    return n_max,n_min

In [18]:
# n_max, n_min값 구하기
n_max,n_min = cal_nmax_nmin()
print(n_max,n_min)  

[1.44, 1.25, 1.12, 6.9, 6.45, 4.53] [1.14, 1.25, 0.97, 4.43, 5.1, 4.53]


### Step 3    
- n(k) = nmin(k) + alpha*(nmax(k) - nmnin(k))   

In [23]:
# n(k) 계산 함수
def cal_nk(alp, n_max, n_min): #n_min이랑 n_max 위에서 받아온 전역변수로 된것같은데,, 이렇게 하면 나중에 문제 생길 수도 있음  
    #print(n_min) 
    #print(n_max)
    
    n = []
    alpha = alp
    for k in range(len(P)):
        n.append(n_min[k] + alp*(n_max[k]-n_min[k]))
        
    return n

In [25]:
#alpha=0, alpha=1일 때 n(k)값
nk_0 = cal_nk(0, n_max, n_min)  
nk_1 = cal_nk(1, n_max, n_min)
print(nk_0)
print(nk_1)

[1.14, 1.25, 0.97, 4.43, 5.1, 4.53]
[1.44, 1.25, 1.12, 6.9, 6.45, 4.53]


### Step 4  

In [27]:
#STEP4 n(k)값에 따른 요일 분배

def create_schedule(alp, n_max, n_min):
    nk = cal_nk(alp, n_max, n_min)    
    
    Mon, Tue, Wed, Thu, Fri = [],[],[],[],[]
    for k in range(len(nk)):
        nk[k] = round(nk[k],0)
        if nk[k] >= 5:
            Mon.append(k+1)
            Tue.append(k+1)
            Wed.append(k+1)
            Thu.append(k+1)
            Fri.append(k+1)
        elif nk[k] == 4:
            Mon.append(k+1)
            Tue.append(k+1)
            Thu.append(k+1)
            Fri.append(k+1)
        elif nk[k] == 3:
            Mon.append(k+1)
            Wed.append(k+1)
            Fri.append(k+1)
        elif nk[k] == 2:
            Tue.append(k+1)
            Thu.append(k+1)
        elif nk[k] == 1:
            Wed.append(k+1)
            
    return nk, Mon, Tue, Wed, Thu, Fri

In [102]:
#alpha = 0 일 때의 요일별 vendor 방문 스케쥴
alp = 0  
nk, Mon, Tue, Wed, Thu, Fri = create_schedule(alp, n_max, n_min)  
print(nk)
print(Mon)
print(Tue)
print(Wed)  
print(Thu)
print(Fri)

[1.0, 1.0, 1.0, 4.0, 5.0, 5.0]
[4, 5, 6]
[4, 5, 6]
[1, 2, 3, 5, 6]
[4, 5, 6]
[4, 5, 6]


### Step 5

In [101]:
#STEP5_CW돌려서 schedule에 따른 총비용 구해주기
# 요일별로 CW 돌려야 함
# full matrix에서 step 4에서 나온 요일별 스케줄 이용해서 자른 다음에 half_mat으로 바꿔서 넣어줘야함  

#monday   
#Mon의 앞이랑 뒤에 0 또 붙여줘야함!!
half_mat_Mon = to_half_mat_by_day(full_mat, Mon)
half_mat_Mon = np.array(half_mat_Mon, dtype = int)
print(type(half_mat_Mon[0,0]))
net_saving_mat = calculate_net_saving(half_mat_Mon, len(half_mat_Mon))
print(net_saving_mat)



AttributeError: 'int' object has no attribute 'append'

In [86]:
def to_half_mat_by_day(full_mat, day): 
    #요일별로 full matrix 잘라서 half_matrix 형태로 바꿔주기  
    # ex) half_mat = np.array([[-1,10,12,15,7],[-1,-1,5,12,11],[-1,-1,-1,7,9],[-1,-1,-1,-1,10],[-1,-1,-1,-1,-1]])
    
    #cut by indices
    indices = np.array([value - 1 for value in day])
    tmp_mat = np.copy(full_mat[indices]) #cut row
    tmp_mat = np.copy(tmp_mat[:,indices]) #cut column
    
    #change to half matrix 
    half_mat = np.triu(tmp_mat)
    #print(half_mat)     
    for i in range(len(half_mat)):
        for j in range(len(half_mat)):
            if half_mat[i,j] == 0: #tmp_mat의 upper diagonal이 모두 0이 아닐 경우에만 이게 맞는 코딩임  
                half_mat[i,j] = -1
    #print(half_mat)

    
    return half_mat

### Step 7 & 8   

In [ ]:
#STEP7&8 alpha가 0에서 1까지 0.1씩 증가하는 경우

mincost = 10000
best_schedule = 0
for alpha in range(11):
    create_schedule(alpha/10)
    if travel_cost < mincost:
        mincost = travel_cost
        best_schedule = create_scedule(alpha/10)
        
return best_schedule, mincost

In [21]:
best_schedule = 0
best_schedule = create_schedule(0)
best_schedule

([1.0, 1.0, 1.0, 4.0, 5.0, 5.0],
 [4, 5, 6],
 [4, 5, 6],
 [1, 2, 3, 5, 6],
 [4, 5, 6],
 [4, 5, 6])